In [7]:
import os, sys,math,folium
import numpy as np
import pandas as pd
import seaborn as sns
%matplotlib inline
import matplotlib.pyplot as plt
from datetime import datetime
plt.style.use('seaborn')
from folium.plugins import HeatMap
from geopy import geocoders
from geopy.geocoders import GoogleV3
from folium.plugins import MarkerCluster
import bs4,re,requests

In [144]:
sites=["http://forums.redflagdeals.com/cell-phones-f88/"]

brands=["Freedom-Mobile","Koodo-Mobile","Rogers","Fido","Virgin-Mobile","Telus",
        "Bell","Public-Mobile","chatr-wireless"]

brand=[]
plan=[]
price=[]
minutes=[]
data=[]
deals=[]

for i in brands:
    whistleScrape(i)
    print(i + " done")
    
df=pd.DataFrame({"brand":brand,"plan":plan,"price":price,"minutes":minutes,"data":data})

df['data']=df['data'].apply(lambda x: 0 if x=="No" else x)
df['data']=df['data'].apply(lambda x: x.replace('GB','') if 'G' in str(x) else x) 
df['data']=df['data'].apply(lambda x: int(x.replace("MB",''))/1000 if 'M' in str(x) else x)

df.to_csv('data/compOverview.csv')

Freedom-Mobile done
Koodo-Mobile done
Rogers done
Fido done
Virgin-Mobile done
Telus done
Bell done
Public-Mobile done
chatr-wireless done


In [111]:
def whistleScrape(brandName):
    path="https://www.whistleout.ca/CellPhones/Carriers/"+brandName
    res=requests.get(path)
    soup=bs4.BeautifulSoup(res.text,'html.parser')
    pNum=getLength(soup)
    for i in range(pNum): brand.append(brandName)
    getBrandPlan(soup,pNum)
    #getDeal(soup,pNum)
    getMinutes(soup,pNum)
    getPrice(soup,pNum)
    getData(soup,pNum)
    
        
def getBrandPlan(soup,pNum):
    rows=soup.select("h2")
    for i in range(1,pNum+1):
        temp=str(rows[i+1]).split("/span>")[2]
        temp=temp.split("<")[0].strip()
        plan.append(temp)
        
def getDeal(soup,pNum):
    rows=soup.find_all('div',{'class':'position-relative'})
    for i in range(pNum):
        if 'c-green' in str(rows[i]):
            print('g')
            temp=str(rows[i]).split("Offer!")[1].split("data-ht")[0][:-1]
            deals.append(temp)
        else:
            deals.append(0)
        
def getMinutes(soup,pNum):
    rows=soup.find_all("div",{'class':'font-600'})
    for i in range(pNum):
        temp=str(rows[i*4])
        temp=temp.split(">")[1].split("<")[0].strip()
        minutes.append(temp)
        
def getPrice(soup,pNum):
    rows=soup.find_all("strong",{'class':'product-price'})
    for i in range(pNum):
        temp=str(rows[i]).split("$")[1].split('</span>')[1].split('<')[0]
        price.append(temp)
        
def getData(soup,pNum):
    rows=soup.find_all("div",{'class':'font-600'})
    for i in range(pNum):
        temp=str(rows[i*4+2])
        temp=temp.split(">")[1].split("<")[0].strip()
        data.append(temp)
            
def getLength(soup):
    rows=soup.find_all('h3',{'class':'font-800'})
    temp=str(rows[0]).split('offers ')[1].split(' plans')[0]
    return int(temp)
            


In [142]:

df.data

0         0
1      0.25
2         2
3        10
4         5
5        10
6        15
7        12
8        20
9        20
10        0
11        0
12     0.25
13        0
14        0
15        0
16        1
17      0.1
18        0
19      0.5
20        0
21        0
22        0
23        5
24        2
25      0.5
26        0
27        5
28        6
29        1
       ... 
566       3
567       3
568     1.5
569     1.5
570      12
571       6
572       6
573       3
574       3
575     1.5
576      18
577      12
578       9
579       6
580       6
581       3
582      18
583      12
584       9
585       6
586      18
587      15
588       9
589      15
590      15
591       0
592       0
593       0
594       0
595       0
Name: data, Length: 596, dtype: object